# Get Nike locations

#### Load Python tools and Jupyter config

In [1]:
import us 
import json
import black
import requests
import pandas as pd
import jupyter_black
import altair as alt
import geopandas as gpd
from bs4 import BeautifulSoup
from vega_datasets import data
from tqdm.notebook import tqdm, trange

In [2]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [3]:
place = "nike"
place_formal = "Nike"
color = "#ff6600"
latitude = "39.106667"
longitude = "-94.676392"

## Read data

#### All the locations

In [4]:
headers = {
    "authority": "api.nike.com",
    "content-type": "application/json; charset=UTF-8",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
}

response = requests.get(
    "https://api.nike.com/store/store_locations/v1?fields=storeNumber,storeServices,id,name,address,businessConcept,company,coordinates,distance,facilityType,imageUrl,localizations,storeConcept,offerings,operationalDetails,phone,region,slug,timezone,defaultLanguage&search=(facilityType%20=in=%20(%20%27NIKE_OWNED_STORE%27))%20and%20(businessConcept%20=out=%20(%27EMPLOYEE_STORE%27,%20%27BRAND_POPUP%27))%20and%20(brand%20!=%20%27CONVERSE%27)%20and%20(coordinates=geoProximity=%7B%22latitude%22%3A34%2C%22longitude%22%3A-118.43%2C%22maxDistance%22%3A50000%2C%22measurementUnits%22%3A%22mi%22%7D)&count=2000",
    headers=headers,
)

In [5]:
src = pd.DataFrame(response.json()["objects"]).drop(
    ["storeServices", "offerings", "imageUrl", "id", "businessConcept", "distance"],
    axis=1,
)

In [6]:
src[
    [
        "address1",
        "address2",
        "area",
        "city",
        "state",
        "postalCode",
        "country",
        "iso2Country",
        "address3",
        "county",
    ]
] = pd.json_normalize(src["address"])

In [7]:
src["open_date"] = pd.json_normalize(src["operationalDetails"])["openingDate"]

In [8]:
src[["latitude", "longitude"]] = pd.json_normalize(src["coordinates"])

In [9]:
df = (
    src.query("country == 'USA'")[
        [
            "storeNumber",
            "facilityType",
            "phone",
            "name",
            "region",
            "company",
            "address1",
            "city",
            "state",
            "postalCode",
            "country",
            "open_date",
            "latitude",
            "longitude",
        ]
    ]
    .rename(columns={"address1": "street"})
    .copy()
)

#### Create a mapping of state abbreviations to full state names using the us library

In [10]:
state_mapping = {state.abbr: state.name for state in us.states.STATES}

#### New column of full state names based on abbreviations

In [11]:
df["state_name"] = df["state"].map(state_mapping)

In [12]:
len(df)

295

---

## Geography

#### Make it a geodataframe

In [13]:
df_geo = df.copy()

In [14]:
gdf = gpd.GeoDataFrame(
    df_geo, geometry=gpd.points_from_xy(df_geo.longitude, df_geo.latitude)
)

---

## Maps

#### US states background

In [15]:
background = (
    alt.Chart(alt.topo_feature(data.us_10m.url, feature="states"))
    .mark_geoshape(fill="#e9e9e9", stroke="white")
    .properties(width=800, height=500, title=f"{place_formal} locations")
    .project("albersUsa")
)

#### Location points map

In [16]:
points = (
    alt.Chart(gdf)
    .mark_circle(size=10, color=color)
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
    )
)

point_map = background + points
point_map.configure_view(stroke=None)

alt.LayerChart(...)

#### Location proportional symbols map

In [17]:
symbols = (
    alt.Chart(gdf)
    .transform_aggregate(
        latitude="mean(latitude)",
        longitude="mean(longitude)",
        count="count()",
        groupby=["state"],
    )
    .mark_circle()
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
        size=alt.Size("count:Q", title="Count by state"),
        color=alt.value(color),
        tooltip=["state:N", "count:Q"],
    )
    .properties(
        title=f"Number of {place_formal} in US, by average lon/lat of locations"
    )
)

symbol_map = background + symbols
symbol_map.configure_view(stroke=None)

alt.LayerChart(...)

---

## Exports

#### JSON

In [18]:
df.to_json(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.json",
    indent=4,
    orient="records",
)

#### CSV

In [19]:
df.to_csv(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.csv", index=False
)

#### GeoJSON

In [20]:
gdf.to_file(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.geojson",
    driver="GeoJSON",
)